In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
keras = tf.keras


2023-03-14 12:40:44.609475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
test = tf.data.Dataset.from_tensor_slices([1, 2, 3])

In [6]:


import sys
sys.path.append("..")

from mechanism.mechanized_sequential import MechanizedSequential
from mechanism.mechanized_sequential import Mechanism



In [40]:
Q_MEAN = 0.5
EPOCH = 5
POPULATION_SIZE = 10000
TRAIN_DIM = 1000
BATCH_SIZE = 5

In [41]:



def initialize_with_str_seed(init_str):
    """
    Initializes random number generator with seed corresponding to given input string init_str.
    :param init_str: Initialization string according to which seed will be computed. Seed is the sum of the ASCII
                     values of each character in init_str.
    """
    rnd_val = 0
    if init_str:
        for c in init_str:
            rnd_val += ord(c)
    np.random.seed(rnd_val)

def gen_data(n, d, seed = None):
    if seed:
        initialize_with_str_seed(seed)
    p = (1.0 + np.sqrt(max(2 * Q_MEAN - 1, 1 - 2 * Q_MEAN))) / 2 
    data = np.random.choice([-1, 1], (n, d), p=[1 -p, p])
    data_y = np.random.choice([0, 1], n, p=[1 -p, p])
    return data, data_y

def gen_valid(n, d, seed = None):
    if seed:
        initialize_with_str_seed(seed)
    
    n = int(n/10)
    
    p = (1.0 + np.sqrt(max(2 * Q_MEAN - 1, 1 - 2 * Q_MEAN))) / 2 
    data = np.random.choice([-1, 1], (n, d), p=[1 -p, p])
    data_y = np.random.choice([0, 1], n, p=[1 -p, p])
    return data, data_y



In [31]:
import math

x_train, y_train = gen_data(POPULATION_SIZE, TRAIN_DIM)
x_valid, y_valid = gen_valid(POPULATION_SIZE, TRAIN_DIM)


In [42]:

def compile_and_fit_model(model, train_set, eager = False):
    keras.backend.clear_session()
    tf.random.set_seed(42)
    np.random.seed(42)
    
    lr_schedule = keras.callbacks.LearningRateScheduler(
        lambda epoch: 1e-8 * 10**(epoch / 20))
    optimizer = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
    model.compile(loss=keras.losses.Huber(),
                optimizer=optimizer,
                metrics=["mae"],
                run_eagerly = eager)

    history = model.fit(train_set, epochs = EPOCH, callbacks=[lr_schedule])
    return history

def eval_model(train_size, mechanism = None, sigma = 0.1, hold_frac = 0.4, threshold = 0.5):
    ''' Compile and fit the empirical model as baseline'''
    model = MechanizedSequential([
        keras.layers.Conv1D(filters = 32,
                            kernel_size = 5,
                            strides = 1, 
                            padding = "causal",
                            activation = "relu",
                            input_shape = [None, 1]),
        keras.layers.LSTM(32, return_sequences = True),
        keras.layers.LSTM(32, return_sequences=True),
        keras.layers.Dense(1),
        keras.layers.Lambda(lambda x: x * 200)
    ])    
    # batch_size = math.floor(POPULATION_SIZE/step)
    train_set = tf.data.Dataset.from_tensor_slices((x_train[:train_size], y_train[:train_size])).batch(BATCH_SIZE)


    if mechanism:
        model.choose_mech(mechanism)
        model.set_mechanism_para(sigma = sigma, hold_frac = hold_frac, threshold = threshold)
        history = compile_and_fit_model(model, train_set, True)
    else:
        history = compile_and_fit_model(model, train_set, False)


    ''' Validate the result'''
    # x_valid, y_valid = gen_valid(POPULATION_SIZE, TRAIN_DIM)
    model.choose_mech(None)
    predict = model.predict(x_valid)
    
    error = generalization_error(y_valid, predict)
    return history, predict, error


def generalization_error(true_val, predict_val):
    error = keras.metrics.RootMeanSquaredError()
    error.update_state(true_val, predict_val)
    return error.result().numpy()


def eval_multiple_rounds(rounds, mechanism = None, sigma = 0.1, hold_frac = 0.4, threshold = 0.5):
    history_list, predict_list, generalization_error_list = [], [], []
    for r in rounds:
        history, predict, error = eval_model(train_size = r, mechanism = mechanism, sigma = sigma, hold_frac = hold_frac, threshold = threshold)
        history_list.append(history)
        predict_list.append(predict)
        generalization_error_list.append(error) 

    return history_list, predict_list, generalization_error_list




# Evaluate the Model

In [44]:


'''
Plot the Comparison of the Three Model
'''
def plot_error(rounds, generalization_error, mechanism):
    plt.plot(rounds, generalization_error, label = mechanism)
    plt.xlabel("Queries")
    plt.ylabel("RMSE (Generalization Error) for adaptive queries")
    plt.legend()
    plt.grid()

max_rounds = 100

stepped_rounds = range(math.floor(max_rounds/2), max_rounds, 10)

''' eval the empirical model as baseline'''





' eval the empirical model as baseline'

In [45]:
baselin_history_list, baseline_predict_list, baseline_generalization_error_list = eval_multiple_rounds(stepped_rounds)
print(baseline_generalization_error_list)

Epoch 1/5
10/10 [==============================] - 7s 376ms/step - loss: 15.5776 - mae: 16.0773 - lr: 1.0000e-08
Epoch 2/5
10/10 [==============================] - 4s 379ms/step - loss: 15.4346 - mae: 15.9343 - lr: 1.1220e-08
Epoch 3/5
10/10 [==============================] - 4s 381ms/step - loss: 15.2183 - mae: 15.7180 - lr: 1.2589e-08
Epoch 4/5
10/10 [==============================] - 4s 383ms/step - loss: 14.9561 - mae: 15.4558 - lr: 1.4125e-08
Epoch 5/5
32/32 [==============================] - 4s 102ms/step


Epoch 1/5
11/11 [==============================] - 9s 491ms/step - loss: 5.7584 - mae: 6.2421 - lr: 1.0000e-08
Epoch 2/5
11/11 [==============================] - 5s 442ms/step - loss: 5.7324 - mae: 6.2160 - lr: 1.1220e-08
Epoch 3/5
11/11 [==============================] - 5s 429ms/step - loss: 5.6945 - mae: 6.1780 - lr: 1.2589e-08
Epoch 4/5
11/11 [==============================] - 5s 437ms/step - loss: 5.6506 - mae: 6.1338 - lr: 1.4125e-08
Epoch 5/5
32/32 [==============================] - 4s 110ms/step


Epoch 1/5
11/11 [==============================] - 8s 451ms/step - loss: 6.4464 - mae: 6.9342 - lr: 1.0000e-08
Epoch 2/5
11/11 [==============================] - 5s 468ms/step - loss: 6.3494 - mae: 6.8368 - lr: 1.1220e-08
Epoch 3/5
11/11 [==============================] - 5s 404ms/step - loss: 6.2083 - mae: 6.6951 - lr: 1.2589e-08
Epoch 4/5
11/11 [==============================] - 8s 783ms/step - loss: 6.0444 - mae: 6.5305 - lr: 1.4125e-08
Epoch 5/5
32/32 [==============================] - 5s 141ms/step


Epoch 1/5
12/12 [==============================] - 9s 461ms/step - loss: 10.3493 - mae: 10.8492 - lr: 1.0000e-08
Epoch 2/5
 7/12 [================>.............] - ETA: 2s - loss: 10.3297 - mae: 10.8295

KeyboardInterrupt: 

In [ ]:



baselin_history_list, baseline_predict_list, baseline_generalization_error_list = eval_multiple_rounds(stepped_rounds)
print(baseline_generalization_error_list)

# baseline_generalization_error_list = [41.741657, 34.947556, 29.856426, 26.357683, 24.293772]

''' Compile and fit the gaussian model'''
gaussian_history_list, gaussian_predict_list, gaussian_generalization_error_list = eval_multiple_rounds(stepped_rounds, mechanism = Mechanism.MechanismType.GAUSSIAN, sigma = 0.03)
print(gaussian_generalization_error_list)

# gaussian_generalization_error_list = [65.59238, 53.909763, 41.66362, 32.519978, 26.558846]

''' Compile and fit the threshold out model'''
threshold_history_list, threshold_predict_list, threshold_generalization_error_list = eval_multiple_rounds(stepped_rounds, mechanism = Mechanism.MechanismType.THRESHOLD, sigma = 0.1, hold_frac = 0.4, threshold = 0.5)
print(threshold_generalization_error_list)

# threshold_generalization_error_list = [15.81994, 19.699022, 23.081583, 25.196918, 26.020424]

""" plot the generalization error """
plt.figure()
plot_error(stepped_rounds, baseline_generalization_error_list, "Baseline")
plot_error(stepped_rounds, gaussian_generalization_error_list, "Gaussian - AdaptFun")
plot_error(stepped_rounds, threshold_generalization_error_list, "Threshold")
plt.show()